In [1]:
from mtcnn import MTCNN
import cv2
import torch
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [8]:
from mtcnn.mtcnn import MTCNN
import cv2
import os

# Initialize the MTCNN detector
detector = MTCNN()

# Path to your image folder
image_folder = 'Database_of_Images'

# List to store detected faces
faces = []

# Loop through images and detect faces
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder, filename))
    
    # Detect faces using the MTCNN detector
    result = detector.detect_faces(img)
    
    if result:
        for face_info in result:
            x, y, width, height = face_info['box']
            face = img[y:y+height, x:x+width]
            faces.append(face)


1/1 [==============================] - 0s 22ms/step


In [13]:

# Initialize the pre-trained InceptionResnetV1 model
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# Define the transformation to apply to your face images
transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])

encodings = []
for image in faces:
    image = transform(image)
    image = image.unsqueeze(0)
    image = np.around(image, decimals=12)
    encoding = resnet(image)
    encoding /= torch.norm(encoding, p=2)
    encodings.append(encoding)
# database = {}
# database["Salman Khan"] = img_to_encoding("Database_of_Images/OIP.jpg", FRmodel)
# database["Kareena Kapoor"] = img_to_encoding("Database_of_Images/R (1).jpg", FRmodel)
# database["Katrina Kaif"] = img_to_encoding("Database_of_Images/R (2).jpg", FRmodel)
# database["Shahrukh Khan"] = img_to_encoding("Database_of_Images/R.jpg", FRmodel)
# database["Shaheen Afridi"] = img_to_encoding("Database_of_Images/Shaheen-Afridi-2.jpg", FRmodel)

In [14]:
new_img = cv2.imread('karena.jpg')
new_result = detector.detect_faces(new_img)


# Check if a face was detected
if new_result:
    # Get the coordinates of the first detected face
    x, y, width, height = new_result[0]['box']

    # Extract the face region from the image
    detected_face = new_img[y:y+height, x:x+width]

    # Now, 'detected_face' contains the pixel values of the detected face
else:
    print("No face detected in the image.")


1/1 [==============================] - 0s 16ms/step


In [15]:
# Display the detected face using OpenCV
cv2.imshow('Detected Face', detected_face)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()  # Close the OpenCV window

In [18]:

image_new = transform(detected_face)
image_new = image_new.unsqueeze(0)
image_new = np.around(image_new, decimals=12)
new_encoding = resnet(image_new)
new_encoding /= torch.norm(new_encoding, p=2)

In [21]:
import torch
import numpy as np

# Assuming you have already defined 'target_encoding' and 'encodings' correctly.

# Set a threshold for matching
threshold = 1.2  # You can adjust this based on your needs

# Initialize variables for minimum distance and identity
min_dist = float('inf')

# Initialize match as False
match = False

# Calculate the distance between the target encoding and each reference encoding
for ref_encoding in encodings:
    dist = torch.norm(new_encoding - ref_encoding).detach().numpy()
    print(dist)
    # Check if the current distance is less than the threshold
    if dist <= threshold:
        match = True  # Match found
        break

if match:
    print(f"Match found.")
else:
    print("No match found.")


1.3418177
1.1324469
Match found.
